In [1]:
import syft as sy
import torch
import pandas as pd

In [2]:
duet = sy.launch_duet(loopback=True)
#duet=sy.launch_duet(network_url="http://ec2-3-15-165-254.us-east-2.compute.amazonaws.com:5000/")

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...


/home/iwan/anaconda3/envs/pysft_tutorial/lib/python3.9/site-packages/aiortc/rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
/home/iwan/anaconda3/envs/pysft_tutorial/lib/python3.9/site-packages/aiortc/rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!



In [3]:
df_all = pd.read_csv('data/train2.csv')

In [4]:
features = df_all.columns[2:]
target = 'label'

In [5]:
df_all.sample(5)

,label,filename,0,1,2,3,4,5,6,7,...,490,491,492,493,494,495,496,497,498,499
1654,1,Data/12k_DE/B014_2.mat,-1.129686,-0.713924,-0.152021,-0.643686,-0.998273,-0.298161,0.400818,-0.052329,...,1.303713,-1.713113,-0.582511,1.673028,-0.415980,-2.191183,0.058692,1.435126,-0.438637,-1.060581
399,3,Data/12k_DE/OR021@6_3.mat,0.186159,0.638737,0.176715,-0.286034,-0.150914,-0.191595,-0.215568,-0.046305,...,-0.207577,-0.166169,-0.090619,-0.028144,0.084456,0.027793,-0.025965,0.216669,0.306023,0.142572
3009,2,Data/12k_DE/IR007_1.mat,0.623086,-0.049763,-0.273862,0.980312,-0.990533,-2.108805,1.837322,2.496304,...,-0.641627,-0.217283,0.033996,-0.119101,-0.103015,0.015136,-0.186219,-0.495741,-0.352074,0.180991
450,1,Data/12k_DE/B028_2.mat,0.510771,0.411781,-0.428686,-1.066242,0.146670,1.608384,0.056782,-2.036420,...,-0.499231,0.987895,0.462982,-0.778754,-0.374830,0.259693,0.056403,0.091675,0.102295,-0.066102
85,3,Data/12k_DE/OR007@3_0.mat,2.257598,-0.536102,-2.386605,-0.494933,1.787847,0.781313,-1.374793,-1.077636,...,0.152163,0.295199,-0.172969,-0.261641,0.314200,0.641443,0.104660,-0.481210,-0.135494,0.573355


In [6]:
df_all.shape

(3502, 502)

In [7]:
X = torch.tensor(df_all[features].values, dtype=torch.float32)
y = torch.tensor(df_all[target].values, dtype=torch.long)

In [8]:
print(X.shape)

torch.Size([3502, 500])


In [9]:
ds = [t for t in zip(X, y)]
ds = sy.lib.python.List(ds)

In [10]:
X = X.tag("bearing-data")
y = y.tag("bearing-target")
ds = ds.tag("bearing-tensordata")

X = X.describe(
    f"This is a dataset for bearing fault classification of {df_all.shape[0]} samples. 500 Features are machine sequence"
)
y = y.describe("Labels for bearing fault classification: Normal (N), Fault at Ball (B), Fault at Inner Raceway (IR), Fault at Outer Raceway (OR)")
ds = ds.describe("Dataset wrapper for both target and input")

In [11]:
data_pointer = X.send(duet, pointable=True)
target_pointer = y.send(duet, pointable=True)

In [12]:
ds = ds.send(duet, pointable=True)

In [13]:
duet.store

[<syft.proxy.torch.TensorPointer object at 0x7fe218d200a0>, <syft.proxy.torch.TensorPointer object at 0x7fe218d20310>, <syft.proxy.syft.lib.python.ListPointer object at 0x7fe218d20520>]

In [14]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: 1b70cb32d2424ff2a19392dc0a6cc552>,[bearing-data],This is a dataset for bearing fault classifica...,<class 'torch.Tensor'>
1,<UID: 469112699e8149f3b235eaf96a87393d>,[bearing-target],Labels for bearing fault classification: Norma...,<class 'torch.Tensor'>
2,<UID: c266b82921394f2293e5eab7024d568a>,[bearing-tensordata],Dataset wrapper for both target and input,<class 'syft.lib.python.list.List'>


# Request Access

In [15]:
duet.requests.add_handler(
    action="accept",
    print_local=True
)

[2021-12-30T17:51:49.882730+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: fe3c9be5ba22433789f0c5cfb048904d>: To run test and inference locally
Value: <Storable: True>
[2021-12-30T17:52:09.111425+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: afc8d55dbef248f89440166e875009e6>:
Value: <Storable: 3502>
[2021-12-30T17:53:14.063392+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: e8851eaa53ee40109d793007a35e4298>: To run test and inference locally
Value: <Storable: True>
[2021-12-30T17:53:33.470479+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: ee8a225950e1495aaaa340ec7d257b5c>:
Value: <Storable: 3502>
[2021-12-30T17:53:52.430799+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: 87a73e09ed974aa8a2633cc263bd7c51>:
Value: <Storable: 55>
[2021-12-30T17:53:53.537879+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: fd83127b58a94ef8a6ee0a4cf9ad5a51>: To evaluate training progress
Value: <Storable: 1.4355052709579468>
[2021-12-30T17:53:55.424012+0900

[2021-12-30T17:54:21.614727+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: c44432ac0f6f4c828eb4dca7502cbf4f>: To evaluate training progress
Value: <Storable: '64, eps=9.999999747378752e-06, momentum=0.10000000149011612, affine=True, track_running_stats=True'>
[2021-12-30T17:54:22.716770+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: e075411b724e4eebbf86fc2f5af0db79>: To evaluate training progress
Value: <Storable: OrderedDict([('weight', tensor([1.0193, 0.9534, 1.0646, 1.0348, 0.9451, 1.1340, 1.0577, 1.0581, 1.0934,    0.9773, 1.0330, 0.9785, 1.0360, 0.9435, 0.9332, 1.0581, 1.0188, 1.1636,    0.8476, 0.9757, 1.0430, 0.8698, 0.8905, 1.0555, 1.0895, 0.9839, 1.0662,    1.0898, 1.0603, 0.7770, 1.0579, 1.0576, 1.0746, 1.0348, 0.9075, 1.0755,    0.9147, 0.8644, 1.0579, 1.0226, 0.9992, 0.9539, 0.9814, 0.9316, 0.8763,    0.8938, 0.9789, 0.9356, 0.9036, 1.0156, 0.9076, 0.8785, 0.9442, 0.8734,    0.9488, 1.0237, 1.0655, 0.9714, 1.0550, 0.9671, 1.1696, 0.9386, 1.0163,    1.0308]

[2021-12-30T17:54:32.696897+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: 8dc16f86df314d218efd6c25ff53a904>: To evaluate training progress
Value: <Storable: 'p=0.5, inplace=False'>
[2021-12-30T17:54:33.785852+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: 3b4a79fa6f5c422daee2cf429321be1c>: To evaluate training progress
Value: <Storable: OrderedDict()>
[2021-12-30T17:54:34.896714+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: 84c1f5c970ca4fa4852122229df56756>: To evaluate training progress
Value: <Storable: 'kernel_size=(2,), stride=(2,), padding=(0,)'>
[2021-12-30T17:54:36.005571+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: ff3435b467824d5498154bfcb446c499>: To evaluate training progress
Value: <Storable: OrderedDict()>
[2021-12-30T17:54:37.117500+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: e7561345759c470c940e57703a4d1a34>: To evaluate training progress
Value: <Storable: 'in_features=16000, out_features=4, bias=True'>
[2021-12-30T17:5

[2021-12-30T17:55:49.654539+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: 597a1aad4f9b4c4d8f20d881dc98845d>: To evaluate training progress
Value: <Storable: '64, eps=9.999999747378752e-06, momentum=0.10000000149011612, affine=True, track_running_stats=True'>
[2021-12-30T17:55:50.750004+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: 82c4f39f78b04d82a01b99b29560a21d>: To evaluate training progress
Value: <Storable: OrderedDict([('weight', tensor([1.0007, 0.9038, 0.9760, 1.0668, 0.9321, 1.2206, 0.9193, 0.9761, 1.0527,    1.0189, 0.9210, 0.9969, 0.9021, 0.8718, 0.9189, 0.9243, 0.9474, 1.1312,    0.8242, 0.8098, 0.9597, 0.8377, 0.8634, 0.9496, 0.9331, 0.9521, 1.1806,    1.0367, 0.8574, 0.8055, 1.0375, 0.9652, 1.0400, 0.8940, 0.8614, 1.1360,    0.8099, 0.8177, 1.1013, 1.0992, 1.0529, 0.9556, 0.8427, 0.8578, 0.8968,    0.7735, 0.8846, 1.0503, 0.9848, 0.8383, 0.8942, 0.8739, 0.8166, 0.9456,    0.9048, 0.9090, 0.8657, 0.8732, 1.0601, 0.9460, 1.0994, 0.8295, 0.9470,    0.9088]

[2021-12-30T17:55:59.734336+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: 2759a53b3f0b46959a7a93924172424e>: To evaluate training progress
Value: <Storable: 'p=0.5, inplace=False'>
[2021-12-30T17:56:00.855418+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: 3f3bc2ece8d2453caab28be06b81bf5a>: To evaluate training progress
Value: <Storable: OrderedDict()>
[2021-12-30T17:56:01.953639+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: 5c26388f639c46dc8f41025e2a7ff328>: To evaluate training progress
Value: <Storable: 'kernel_size=(2,), stride=(2,), padding=(0,)'>
[2021-12-30T17:56:03.064882+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: 11d2f768e47347b8831441ed54b73d22>: To evaluate training progress
Value: <Storable: OrderedDict()>
[2021-12-30T17:56:04.163848+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: b9479a0171aa4108bf1640cddd08e760>: To evaluate training progress
Value: <Storable: 'in_features=16000, out_features=4, bias=True'>
[2021-12-30T17:5

...

[2021-12-30T17:58:41.146943+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: c18886270f764e9fb04d203422d8c2de>: To evaluate training progress
Value: <Storable: '64, eps=9.999999747378752e-06, momentum=0.10000000149011612, affine=True, track_running_stats=True'>
[2021-12-30T17:58:41.311413+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: 3f9d1f32c4e34dd9bda4afbfb3e6091a>: To evaluate training progress
Value: <Storable: OrderedDict([('weight', tensor([1.1262, 0.7563, 0.9555, 1.0257, 0.8312, 1.1822, 1.0311, 0.9983, 1.0822,    0.9544, 0.6605, 0.9685, 0.7588, 0.8651, 0.7418, 0.8808, 0.8598, 1.1550,    0.8557, 0.7728, 0.8439, 0.8003, 0.7278, 0.8628, 0.8752, 1.0703, 1.2958,    1.0910, 0.7868, 0.8248, 1.0909, 0.8545, 1.1445, 0.8376, 0.9053, 1.1907,    0.7656, 0.8327, 1.1822, 1.0781, 1.0815, 0.9394, 0.9089, 0.8946, 0.8501,    0.7788, 0.7648, 1.0698, 0.9526, 0.6911, 0.9229, 0.9129, 0.6547, 1.0574,    0.8665, 0.6516, 0.7495, 0.8136, 1.1225, 0.9430, 1.1214, 0.7136, 0.8624,    0.7781]

[2021-12-30T17:58:49.220415+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: c1617cc3029d42b29273692896469c6b>: To evaluate training progress
Value: <Storable: OrderedDict([('weight', tensor([0.6079, 0.5791, 0.6207, 0.4488, 0.5695, 0.7224, 0.6143, 0.4502, 0.6798,    0.5016, 0.6997, 0.5025, 0.4783, 0.6239, 0.7270, 0.6438, 0.5466, 0.5248,    0.5225, 0.6094, 0.5600, 0.7480, 0.5863, 0.5428, 0.4344, 0.4413, 0.4933,    0.4874, 0.6875, 0.4481, 0.4657, 0.4064, 0.5060, 0.5280, 0.5908, 0.5457,    0.5534, 0.5535, 0.6000, 0.5405, 0.3966, 0.4714, 0.6834, 0.6302, 0.5696,    0.6371, 0.7675, 0.6593, 0.4640, 0.4351, 0.5961, 0.5433, 0.6597, 0.7108,    0.4773, 0.6253, 0.6314, 0.6934, 0.5248, 0.7142, 0.4843, 0.7258, 0.5081,    0.4574, 0.6064, 0.6759, 0.5373, 0.4405, 0.5264, 0.5019, 0.5300, 0.6027,    0.5549, 0.4441, 0.7528, 0.6012, 0.6030, 0.5270, 0.8073, 0.7701, 0.6862,    0.5082, 0.6491, 0.5975, 0.5629, 0.4909, 0.5418, 0.7046, 0.4796, 0.5511,    0.6858, 0.5392, 0.6426, 0.6652, 0.7651, 0.3809, 0.5

[2021-12-30T18:00:01.228427+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: 6dc0e474521b498588c6313f2dbda118>: To evaluate training progress
Value: <Storable: OrderedDict([('weight', tensor([[[ 3.8104e-01, -3.8501e-01, 4.7440e-01, -4.0870e-02, -3.2077e-01,      3.4255e-01, 3.0100e-03, -7.3327e-02, 3.4845e-01]],    [[-5.6195e-01, -1.8469e-01, 7.0238e-02, 1.1028e-01, 2.3149e-01,     -1.6227e-01, 1.4711e-01, 3.2958e-01, -6.7921e-03]],    [[-4.7018e-01, -1.3012e-01, 2.1427e-01, -2.4833e-01, 1.0802e-01,      1.3868e-01, -3.3588e-01, -2.3554e-01, -1.0831e-01]],    [[-2.4524e-01, -4.1914e-01, -3.0694e-01, -2.6785e-01, -2.6964e-01,     -2.7574e-01, -1.2523e-02, -3.9352e-01, -1.8183e-01]],    [[ 2.9172e-01, 2.5960e-01, 2.2341e-01, 3.7360e-01, 3.1330e-01,      1.8176e-01, -7.8048e-02, 3.2614e-01, -1.6925e-01]],    [[-5.8880e-01, -1.3806e-02, -3.5821e-01, 2.1007e-01, 1.1076e-01,      8.6801e-02, -1.0530e-01, -1.4922e-01, -5.4984e-01]],    [[-5.6371e-01, -3.5516e-01, -8.6130e-02, -2.4341e-

[2021-12-30T18:00:02.328181+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: 649c5f7b891a4861ae6740fa4b4b9c8f>: To evaluate training progress
Value: <Storable: '64, eps=9.999999747378752e-06, momentum=0.10000000149011612, affine=True, track_running_stats=True'>
[2021-12-30T18:00:03.441764+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: e23e68e5830546efa2ef6e8fe7703102>: To evaluate training progress
Value: <Storable: OrderedDict([('weight', tensor([1.1483, 0.7829, 1.0391, 0.9611, 0.8408, 1.1778, 0.9621, 1.0133, 1.0734,    0.9455, 0.6526, 0.9816, 0.7679, 0.8532, 0.7282, 0.9125, 0.9018, 1.1589,    0.8899, 0.7720, 0.7985, 0.8095, 0.6506, 0.8394, 0.8991, 1.0700, 1.3098,    1.0702, 0.8624, 0.8928, 1.1378, 0.8561, 1.1764, 0.7328, 0.9439, 1.2315,    0.7173, 0.8095, 1.2021, 1.0026, 1.0563, 0.9038, 0.9223, 0.8917, 0.8864,    0.7732, 0.7891, 1.0956, 0.9275, 0.6526, 0.9705, 0.9272, 0.6715, 1.0916,    0.8616, 0.5949, 0.7574, 0.8397, 1.1888, 0.9734, 1.1449, 0.6989, 0.8858,    0.8129]

[2021-12-30T18:00:10.449364+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: fe0148356a494f8bbb63c888556791dd>: To evaluate training progress
Value: <Storable: 'p=0.5, inplace=False'>
[2021-12-30T18:00:11.558331+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: 49fcd673a35d4658b5a2b8c030555386>: To evaluate training progress
Value: <Storable: OrderedDict()>
[2021-12-30T18:00:12.666896+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: 924b2c79de68472f89f56bcb044a96ad>: To evaluate training progress
Value: <Storable: 'kernel_size=(2,), stride=(2,), padding=(0,)'>
[2021-12-30T18:00:13.781210+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: 6ac9d9603ee6483ca04d5739781bd8d9>: To evaluate training progress
Value: <Storable: OrderedDict()>
[2021-12-30T18:00:14.883201+0900][CRITICAL][logger]][25080] > HANDLER Request <UID: 96667af47fc64b5da992f1435d6248e2>: To evaluate training progress
Value: <Storable: 'in_features=16000, out_features=4, bias=True'>
[2021-12-30T18:0